In [ ]:
import tensorflow as tf
from IPython.display import clear_output, display
import sys
sys.path.insert(0, '..')
from celeba_reader import CelebaSequence
import models

#### Prepare training dataset

In [ ]:
paths = {
    'img_path': '/home/maxim/Desktop/celeba/img_align_celeba',
    'attrs_path': '../list_attr_celeba.csv',
    'partition_path': '../list_eval_partition.csv'
}
train_data = CelebaSequence(partition=0, batch=100, **paths)

#### Start training session, build a model and training operations

In [ ]:
sess = tf.InteractiveSession()

images_batch = tf.placeholder(tf.float32, [None, 218, 178, 3])
labels = tf.placeholder(tf.int32, [None])
classifier = models.initial_model(images_batch)

loss = tf.losses.log_loss(labels=labels, predictions=classifier)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())

saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())

To restore a model from a checkpoint, type:
```python
saver.restore(sess, './checkpoints/model.ckpt')
```

#### Train the model

In [ ]:
def train(checkpoint_period, log_period):
    i = 0
    for x, y in train_data:
        sess.run(train_op, feed_dict={images_batch: x, labels: y})
        if i % log_period == 0:
            clear_output()
            display(i)
        if i % checkpoint_period == 0:
            save_path = saver.save(sess, 'checkpoints/model.ckpt')
            print("Model saved in path: %s" % save_path)
        if i == 1630:
            break
        i = i + 1

In [ ]:
train(checkpoint_period=100, log_period=10)

#### Save trained model

In [ ]:
inputs = {
    "images_batch": images_batch,
    "labels": labels
}
outputs = {"predictions": classifier}
tf.saved_model.simple_save(sess, './cut', inputs, outputs)